In [ ]:
Tatenda

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn

from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report

/Users/tatenda/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Cleaning metadata
# In[147]:
metadata=pd.read_csv("movie_metadata_with_score_metacritic.csv", index_col="Unnamed: 0") #reading in metacritic dataset

In [3]:
metadata=metadata.loc[~metadata["metacritic_metascore"].isna()] #removing rows with no metacritic data


In [4]:
metadata=metadata.drop(["homepage","keywords","overview","status","tagline","imdb_metascore"],1) #dropping unnecessary features

In [5]:
metadata=metadata.drop_duplicates() #dropping duplicates
metadata["release_year"]=metadata.release_date.str[0:4].astype(int) #creating new column with year for comparison

In [6]:
# In[131]:

rev_data=pd.read_csv("Revenue.csv") #reading in revenue data
#cleaning revenue data
#Adjusting units
rev_data["Budget"]=rev_data["Budget($M)"]*1000000
rev_data["Worldwide Gross"]=rev_data["Worldwide Gross($M)"]*1000000
rev_data["Domestic Gross"]=rev_data["Domestic Gross($M)"]*1000000
rev_data=rev_data.drop(["Budget($M)","Domestic Gross($M)","Worldwide Gross($M)"],1)

rev_data=rev_data.rename(columns={"Movie":"title"})#renaming column to match metadata
rev_data=rev_data.drop_duplicates()#dropping duplicates

In [7]:
# In[102]:


critic_revenue=metadata.merge(rev_data,on="title") #merging datasets
critic_revenue=critic_revenue.drop_duplicates() #dropping duplicates
critic_revenue=critic_revenue.loc[(critic_revenue["Worldwide Gross"]!=0) | (critic_revenue["revenue"]!=0)] #removing rows with no revenue data
critic_revenue=critic_revenue.loc[(np.abs(critic_revenue.release_year-critic_revenue.Year)<=5)] #removing rows where the years don't match, as this indicates different movies

#Normalizing by the median for merged data set

critic_revenue["budget"]=(metadata["budget"]-metadata["budget"].median())/metadata["budget"].std()
critic_revenue["Worldwide Gross"]=(rev_data["Worldwide Gross"]-rev_data["Worldwide Gross"].median())/rev_data["Worldwide Gross"].std()

critic_revenue["Domestic Gross"]=(rev_data["Domestic Gross"]-rev_data["Domestic Gross"].median())/rev_data["Domestic Gross"].std()
critic_revenue["popularity"]=(metadata["popularity"]-metadata["popularity"].median())/metadata["popularity"].std()
critic_revenue["revenue"]=(metadata["revenue"]-metadata["revenue"].median())/metadata["revenue"].std()
critic_revenue["vote_count"]=(metadata["vote_count"]-metadata["vote_count"].median())/metadata["vote_count"].std()

In [9]:
critic_revenue.isnull().any()

budget                   True
genres                  False
id                      False
original_language       False
original_title          False
popularity               True
production_companies    False
production_countries    False
release_date            False
revenue                  True
runtime                 False
spoken_languages        False
title                   False
vote_average            False
vote_count               True
metacritic_metascore    False
release_year            False
Month                   False
Day                     False
Year                    False
Budget                  False
Worldwide Gross          True
Domestic Gross           True
dtype: bool

In [10]:
#Replace every nan values with 0
critic_revenue.fillna(value=0,axis=1,inplace=True)
critic_revenue.shape

(3256, 23)

In [11]:
#Defining features and target for this dataset based on co-relation
features = ['Budget']
target = ['Worldwide Gross']

In [12]:
#splitting data set into training and test data set in 0.7/0.3
train, test = train_test_split(critic_revenue,test_size=0.30)
train.head()

,budget,genres,id,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,...,vote_average,vote_count,metacritic_metascore,release_year,Month,Day,Year,Budget,Worldwide Gross,Domestic Gross
524,1.299943,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",44564,en,The Dilemma,0.067575,"[{""name"": ""Imagine Entertainment"", ""id"": 23}, ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2011-01-13,0.496678,...,5.2,0.000000,46.0,2011,Jan,14,2011,70000000.0,1.264979,1.426252
2810,-0.212718,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 80, ""nam...",9336,en,Police Academy,1.371858,"[{""name"": ""Warner Bros."", ""id"": 6194}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1984-03-22,1.173992,...,6.5,1.503544,41.0,1984,Mar,23,1984,4500000.0,-0.024668,0.091170
1303,0.425436,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...",14034,en,What's the Worst That Could Happen?,-0.054004,"[{""name"": ""Metro-Goldwyn-Mayer (MGM)"", ""id"": 8...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2001-05-31,0.502490,...,5.1,0.087540,37.0,2001,Jun,1,2001,30000000.0,0.413580,1.227890
2658,-0.472706,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",20322,en,The Brothers,-0.507101,[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2001-03-23,-0.195277,...,6.5,-0.258014,50.0,2001,Mar,23,2001,6000000.0,-0.001179,-0.076762
1473,1.299943,"[{""id"": 35, ""name"": ""Comedy""}]",88042,en,Parental Guidance,0.129499,"[{""name"": ""Twentieth Century Fox Film Corporat...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-12-25,-0.081316,...,5.7,-0.064503,36.0,2012,Dec,25,2012,25000000.0,0.330033,0.432575


In [13]:
#Fill the training and test data with require information
X_train = train[features].dropna()
y_train = train[target].dropna()
X_test = test[features].dropna()
y_test = test[target].dropna()

In [14]:
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingRegressor
model = ensemble.GradientBoostingRegressor()
model.fit(X_train, y_train)

/Users/tatenda/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [15]:
print('Gradient Boosting R squared": %.4f' % model.score(X_test, y_test))

Gradient Boosting R squared": 0.9320


In [18]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test)
model_mse = mean_squared_error(y_pred, y_test)
model_rmse = np.sqrt(model_mse)
print('Gradient Boosting RMSE: %.4f' % model_rmse)

Gradient Boosting RMSE: 0.2755
